In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-02-14


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [6]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [7]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [8]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

In [9]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-02-13,BOS,BRK,1.266667,3.950000,1
1,2024-02-13,OKC,ORL,1.704225,2.200000,1
2,2024-02-13,MIA,MIL,3.600000,1.307692,1
3,2024-02-13,SAC,PHO,2.700000,1.487805,0
4,2024-02-13,MIN,POR,1.277778,3.850000,1
5,2024-02-13,DET,LAL,4.900000,1.192308,0


In [10]:
prev_games.tail()

,date,visitor,local,v_line,l_line,Results
55,2024-02-12,DEN,MIL,1.847458,1.980392,0
56,2024-02-12,NOP,MEM,1.253165,4.100000,1
57,2024-02-12,WAS,DAL,6.750000,1.117647,0
58,2024-02-12,GSW,UTA,1.892857,1.925926,1
59,2024-02-12,MIN,LAC,2.500000,1.555556,1


In [11]:
prev_games = pd.concat([prev_games, last_games])

In [12]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [13]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [14]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [15]:
tables = soup.findAll('tbody', {'class': 'sportsbook-table__body'})
rows = []
for table in tables:
    for row in table.findAll('tr'):
        rows.append(row)
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK').replace('CHA', 'CHO')
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = american_to_decimal(int(odd))
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = american_to_decimal(int(odd))
        idx = 0
        
        games.append(game)
        game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-02-14,ATL,CHO,1.400000,3.050000
1,2024-02-14,NYK,ORL,2.360000,1.617284
2,2024-02-14,MIA,PHI,2.400000,1.602410
3,2024-02-14,CHI,CLE,4.000000,1.263158
4,2024-02-14,IND,TOR,1.602410,2.400000
5,2024-02-14,BRK,BOS,7.500000,1.100000
6,2024-02-14,WAS,NOP,7.250000,1.105263
7,2024-02-14,HOU,MEM,1.740741,2.200000
8,2024-02-14,SAS,DAL,5.550000,1.160000
9,2024-02-14,SAC,DEN,2.700000,1.487805


In [16]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)